In [1]:
import numpy as np
import csv
import pandas as pd
import pygad
from src import  repeat_elements, tot_TCI_multiple_reactors
import matplotlib.pyplot as plt
from scheduleOptimizationForReactorMix import optimize_schedule, capacity_factor_weeks_approach_mix_reactors
import warnings
import time
warnings.filterwarnings('ignore')

In [2]:
# TCI for a mix of reactors

def tot_TCI_multiple_reactors_mix (power_list, interest_rate, num_reactors_list):
    tot_TCI_for_specific_reactor_power_list = []
    
    for i in range(len(power_list)):
        tot_TCI_for_specific_reactor_power = tot_TCI_multiple_reactors (power_list[i], interest_rate, num_reactors_list[i])
        tot_TCI_for_specific_reactor_power_list.append(tot_TCI_for_specific_reactor_power)
        
    return sum(tot_TCI_for_specific_reactor_power_list)

In [3]:
def level_cost_of_energy_reactor_mix( interest_rate, power_list, num_reactors_list,\
    list_of_generated_MWh_per_year_from_all_reactors_per_demand, list_of_sold_electricity_MWh_per_year_from_all_reactors, elec_price,\
        list_of_OM_cost_per_year_all_reactors):
    
    sum_cost = 0 # initialization 
    sum_elec = 0 # initialization 
           
    for year in range( len(list_of_generated_MWh_per_year_from_all_reactors_per_demand)):
        if year == 0:
            cap_cost =  tot_TCI_multiple_reactors_mix(power_list, interest_rate, num_reactors_list)
            OM_cost_per_year = 0
            elec_gen = 0
            revenue = 0
        
        elif year > 0:
         
            cap_cost = 0 
            
            OM_cost_per_year =  list_of_OM_cost_per_year_all_reactors[year-1]
            revenue = elec_price * list_of_sold_electricity_MWh_per_year_from_all_reactors[year-1]
            elec_gen =  list_of_generated_MWh_per_year_from_all_reactors_per_demand[year-1]
        
        sum_cost += (cap_cost + OM_cost_per_year - revenue) / ((1 +interest_rate)**(year) ) 
        sum_elec += elec_gen/ ((1 + interest_rate)**year) 
    
    LCOE =  sum_cost/ sum_elec
    return LCOE

# Initialize the solution

In [ ]:
# power_list = [1000, 900 , 800, 700, 600, 500, 400, 300, 200, 100  ,50,  20, 5, 1]

# The number of reactors for each type (just a random initialization)
# Num_of_each_reactor_type = [1] *len(power_list)
# Num_of_each_reactor_type = [1, 0, 0 ,0 ,10 ,0 ,0, 0 ,0 ,0, 1, 0, 1, 1]

# Num_of_each_reactor_type
# levelization_period_weeks = 52*10
# demand = sum(power_list) 
# interest_rate = 0.06
# elec_price = 0

# power_list_modified = [power_list [i] for i in range(len(Num_of_each_reactor_type)) if Num_of_each_reactor_type[i] != 0]
# Num_of_each_reactor_type_modified = [x for x in Num_of_each_reactor_type if x != 0]


In [ ]:
# long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)

# results = capacity_factor_weeks_approach_mix_reactors(long_list_power  ,levelization_period_weeks, demand)
# MWh_generated_per_year_per_demand_list = results[4]
# MWh_excess_per_year_list = results[5]
# Tot_OM__cost_per_year_list = results[6]



In [4]:
def initial_population_reactors(power_list, demand, sol_per_pop):
    
    total_sum = sum(power_list)
    rough_multiplier_list =  [np.ceil(demand / total_sum)] * len(power_list)
    
    initial_population = [] 
    for i in range(sol_per_pop ) :
        initial_population.append(rough_multiplier_list )
    return initial_population

def on_gen(ga_instance):
    pass
    # print("Generation : ", ga_instance.generations_completed,  ga_instance.best_solution()[0], ga_instance.best_solution()[1])
    # print(ga_instance.generations_completed, ga_instance.population)


In [5]:
def optimize_schedule(power_list,  levelization_period_weeks, demand , interest_rate, capacity_factor_t_min_criteria):
    
    start_time = time.time()
    
    # GA params
    sol_per_pop = int(2*len(power_list))   
    
    

    num_generations = 300
    num_parents_mating =  max( 3, int(sol_per_pop/3))
    num_genes = len(power_list)
    init_range_low = 0
    init_range_high = 2

    parent_selection_type = "rank"
    keep_parents =  max(3, int(sol_per_pop/3))
    gene_type= int
    crossover_type = "single_point"

    mutation_type = "random"
    mutation_percent_genes = 10
    
    initial_pop  = initial_population_reactors(power_list, demand, sol_per_pop)
    allow_dup  = True
    
    def fitness_eq(output_discrepancy):
        return -100 / ( np.abs(output_discrepancy) ) 
        
    
    def fitness_func(ga_instance, solution, solution_idx):
        
        power_list_modified = [power_list [i] for i in range(len(solution)) if solution[i] != 0]
        Num_of_each_reactor_type_modified = [x for x in solution if x != 0]
        
        long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)
        
        capacity_factor_results =   capacity_factor_weeks_approach_mix_reactors(long_list_power  ,levelization_period_weeks, demand)
        MWh_generated_per_year_per_demand_list = capacity_factor_results[4]
        MWh_excess_per_year_list =     capacity_factor_results[5]
        Tot_OM__cost_per_year_list =  capacity_factor_results[6]
        
        capacity_factor_t_min = min(capacity_factor_results[1])
        
        output_lcoe = level_cost_of_energy_reactor_mix( interest_rate,power_list_modified, Num_of_each_reactor_type_modified,\
    MWh_generated_per_year_per_demand_list, MWh_excess_per_year_list, 0,\
        Tot_OM__cost_per_year_list)

        
        # if min_tot_P(power_list , solution, levelization_period_weeks) == expected_out :
        # fitness = 1 / ( np.abs(output - expected_out)  +1)
        
        fitness = fitness_eq(0 -  output_lcoe) # Here I assume the target is a very small number (zero$/MWh)
        
        if capacity_factor_t_min < capacity_factor_t_min_criteria: # Must satsify the criteria
            fitness = -10000
        
        if sum(long_list_power) < capacity_factor_t_min_criteria*demand:
            fitness = -10000
        if sum(long_list_power) > 2*demand :
            fitness = -10000  
        
        return fitness
            
    ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func= fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_percent_genes= mutation_percent_genes,
                       
                       mutation_type=mutation_type,
                       stop_criteria= ["saturate_20"], # 
                       on_generation= on_gen,
                        fitness_batch_size=1,
                        keep_elitism = int(sol_per_pop/5),
                        crossover_probability = 0.7,
                        gene_type = gene_type,
                    
                       initial_population = initial_pop,
                       allow_duplicate_genes=allow_dup )
      
    
    
    ga_instance.run()
    end_time = time.time() 
    
    
    sol, sol_fitness, _ = ga_instance.best_solution()
    print("\n The optimization program runtime is " , np.round( (end_time -start_time), 0), " sec", " & The Number of Generations Passed is ",\
        ga_instance.generations_completed, "...... Fitness value of the best solution = {solution_fitness}".format(solution_fitness=sol_fitness)) 

    
    
    return solution